In [ ]:
def write(json_data: dict, file_name: str):
    with open(file_name, 'w') as f:
        f.write(json.dumps(json_data))

In [ ]:
# Download list of all currencies
import urllib
import json
import time

url = 'https://api.coingecko.com/api/v3/coins/list?include_platform=true'
response = urllib.request.urlopen(url)
currencies = json.loads(response.read())
write(currencies, 'tmp_step_01.json')
print('Downloaded list of ', len(currencies))
time.sleep(1.21)

In [ ]:
# Include only native or ethereum ERC20s

def include(currency: dict) -> bool:
    if len(currency['platforms']) == 0:
        return True
    if 'ethereum' in currency['platforms']:
        return len(currency['platforms']['ethereum']) != 0
    return False

currencies = list(filter(lambda c: include(c) , currencies))

# Change id to coinGeckoId
for currency in currencies:
    currency['coinGeckoId'] = currency['id']
    currency.pop('id', None )

write(currencies, 'tmp_step_02.json')
print('Filtered out unsupported, remaining count', len(currencies))

In [ ]:
# Get image url rank description
import time
import numpy as np
import requests

base_url = 'https://api.coingecko.com/api/v3/coins/'
query_params = '?localization=false&tickers=false&market_data=false&community_data=false&developer_data=false&sparkline=false'

for idx, currency in enumerate(currencies):
    url = base_url + currency['coinGeckoId'] + query_params
    response = urllib.request.urlopen(url)
    info = json.loads(response.read())

    currency['imageUrl'] = info['image']['large']
    currency['rank'] = info['coingecko_rank']

    if info['links']['homepage'][0] is not None:
        currency['link'] = info['links']['homepage'][0]

    if info['links']['twitter_screen_name'] is not None:
        currency['twitter'] = info['links']['twitter_screen_name']

    if info['description']['en'] is not None and len(info['description']['en']) != 0:
        currency['description'] = info['description']['en']

    write(currencies, 'tmp_step_03.json')
    print('\r', 'Downloaded metadata', idx, end='')
    time.sleep(1.21)

In [ ]:
# Download images if needed
from urllib.parse import urljoin, urlparse
import os
import platform
import requests
import shutil

def file_name_for_url(url: str) -> str:
    return urljoin(url, urlparse(url).path).rsplit('/', 1)[-1]

def file_extension_for_url(url: str) -> str:
    return os.path.splitext(file_name_for_url(url))[-1]

def cache_image_file_name(id: str, url: str) -> str:
    return id + file_extension_for_url(url)

headers = requests.utils.default_headers()
default_agent = headers['User-Agent']
headers.update({'User-Agent': default_agent + ' (' + platform.platform() + ')'})
failed = list()

for idx, currency in enumerate(currencies):
    image_name = cache_image_file_name(currency['coinGeckoId'], currency['imageUrl'])
    image_path = 'images/' + image_name
    if os.path.exists(image_path) == False and currency['imageUrl']:
        res = requests.get(url, headers = headers, stream = True)
    if res.status_code == 200:
        with open(image_path, 'wb') as f:
            shutil.copyfileobj(res.raw, f)
            print('\r', 'Downloaded image', image_name, idx, '         ', end='')
    else:
        failed.append(currency['imageUrl'])
    time.sleep(1.21)

print('Failed count', len(failed))

In [ ]:
# Resize images
from PIL import Image
from PIL import ImageColor
import os

for image_name in os.listdir('images/'):
    path = 'images/' + image_name


# for idx, currency in enumerate(currencies):
#     image_name = cache_image_file_name(currency['coinGeckoId'], currency['imageUrl'])
#     exists = os.path.exists('images/' + image_name)
#     print(exists, image_name)

In [ ]:
# Resize images
from PIL import Image
from PIL import ImageColor
import os

for image_name in os.listdir('images/'):
    path = 'images/' + image_name


# for idx, currency in enumerate(currencies):
#     image_name = cache_image_file_name(currency['coinGeckoId'], currency['imageUrl'])
#     exists = os.path.exists('images/' + image_name)
#     print(exists, image_name)